In [6]:
import onnx
import onnxruntime as ort
import numpy as np
import cv2
import time
import os
import cv2
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import torch
import matplotlib.pyplot as plt

HEIGHT = 384
SAVE_VIDEO = False

cap = cv2.VideoCapture('Polyp_1.mp4') # '/home/s/Downloads/Colon.mp4'
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1024) #  1920 1366
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 768) # 1080 768
cv2.namedWindow('polyp', cv2.WINDOW_NORMAL)

if SAVE_VIDEO:
    out_video = cv2.VideoWriter('out_video.avi', cv2.VideoWriter_fourcc(*'XVID'), cap.get(cv2.CAP_PROP_FPS)/3, (740, 768))

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

with open('best2.trt', 'rb') as f:
	runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING))

	engine = runtime.deserialize_cuda_engine(f.read())
	context = engine.create_execution_context()
        
for binding in engine:
	if engine.binding_is_input(binding):
		input_shape = engine.get_binding_shape(binding)
		input_dtype = trt.nptype(engine.get_binding_dtype(binding))
		input_size = np.empty(input_shape, dtype=input_dtype).nbytes
	else:
		output_shape = engine.get_binding_shape(binding)
		output_dtype = trt.nptype(engine.get_binding_dtype(binding))
		output_size = np.empty(output_shape, dtype=output_dtype).nbytes

d_input = cuda.mem_alloc(input_size)
d_output = cuda.mem_alloc(output_size)

h_output = cuda.pagelocked_empty(trt.volume(output_shape), dtype=output_dtype)
    
bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()


while True:
    m1 = time.time()
    success, frame = cap.read()

    #frame = frame[:,:740,:]
    frame = cv2.resize(frame, (HEIGHT, HEIGHT))
    input1 = frame.copy()
    input2 = cv2.resize(input1, (HEIGHT, HEIGHT))
    h_input = frame.reshape(input_shape).astype(input_dtype).copy()

    cuda.memcpy_htod_async(d_input, h_input, stream)
    context.execute_async_v2(bindings, stream.handle, None)
    cuda.memcpy_dtoh_async(h_output, d_output, stream)
    stream.synchronize()

    output = np.squeeze(np.array(h_output.reshape(tuple(output_shape))) * 255).astype('uint8')
    #output = cv2.cvtColor(output, cv2.COLOR_GRAY2BGR)
    
    print(input2.shape)
    print(output.shape)
    img2 = np.zeros_like(input2)
    img2[:, :, 0] = output[:, :, 0] * 0
    img2[:, :, 1] = output[:, :, 1] * 255
    img2[:, :, 2] = output[:, :, 2] * 255
    
    fps = 1 / (time.time() - m1)
    result = cv2.bitwise_or(input2, img2)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(result, "FPS: " + str(int(fps)), (5,20), font, 0.5, (0,255,0), 1, cv2.LINE_AA)

    cv2.imshow("polyp",result)
    
    
    if cv2.waitKey(1) == ord('q'):
        break

    if SAVE_VIDEO:
        out_video.write(frame_copy)

cap.release()
cv2.destroyAllWindows()
        


(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 384, 3)
(384, 